## Este notebook es encargado utilizar el CSV con las prruebas consolidadas para crear y entrenar un modelo

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn import preprocessing
import math
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [17]:
biometrics_df = pd.read_csv("./1_standarized_biometrics.csv")
biometrics_df.head()

,HR,RR,HRV,MicroSiemens,SCR,SCR_MIN,PhaseName,ArousalMean,Aroused,TimeStamp,...,HR-2,HR-1,HRV-4,HRV-3,HRV-2,HRV-1,MicroSiemens-4,MicroSiemens-3,MicroSiemens-2,MicroSiemens-1
0,0.180957,-0.246737,-0.638383,1.604263,0,0,HA_PV,6.07,1,2020-10-25 19:03:44,...,0.180957,0.180957,-0.638383,-0.638383,-0.638383,-0.638383,1.604263,1.604263,1.604263,1.604263
1,0.463350,-0.508771,-0.930989,1.357679,0,0,HA_PV,6.07,1,2020-10-25 19:03:44,...,0.463350,0.463350,-0.930989,-0.930989,-0.930989,-0.930989,1.357679,1.357679,1.357679,1.357679
2,0.419905,-0.468458,-1.085573,1.357679,0,0,HA_PV,6.07,1,2020-10-25 19:03:45,...,0.419905,0.419905,-1.085573,-1.085573,-1.085573,-1.085573,1.357679,1.357679,1.357679,1.357679
3,0.419905,-0.468458,-1.121459,1.111095,0,0,HA_PV,7.31,1,2020-10-25 19:03:46,...,0.419905,0.419905,-1.121459,-1.121459,-1.121459,-1.121459,1.111095,1.111095,1.111095,1.111095
4,0.506795,-0.549084,-1.088333,1.111095,0,0,HA_PV,7.31,1,2020-10-25 19:03:46,...,0.419905,0.419905,-0.638383,-0.930989,-1.085573,-1.121459,1.604263,1.357679,1.357679,1.111095


In [18]:
# X = biometrics_df[['MicroSiemens', 'HR', 'HRV']].values
X = biometrics_df[['MicroSiemens', 'HR', 'HRV', 'MicroSiemens-4', 'MicroSiemens-3', 'MicroSiemens-2', 'MicroSiemens-1', 'HR-4', 'HR-3', 'HR-2', 'HR-1', 'HRV-4','HRV-3','HRV-2','HRV-1']].values
y = biometrics_df['Aroused'].values

In [19]:
X

array([[ 1.60426292,  0.18095678, -0.63838286, ..., -0.63838286,
        -0.63838286, -0.63838286],
       [ 1.35767905,  0.46335022, -0.93098869, ..., -0.93098869,
        -0.93098869, -0.93098869],
       [ 1.35767905,  0.41990507, -1.08557291, ..., -1.08557291,
        -1.08557291, -1.08557291],
       ...,
       [ 0.92410292, -1.00764835, -1.10377383, ..., -1.0792364 ,
        -1.10868131, -1.12585751],
       [ 1.11904139, -0.46657089, -1.00562414, ..., -1.10868131,
        -1.12585751, -1.10377383],
       [ 1.21651063,  0.73582348, -0.90011322, ..., -1.12585751,
        -1.10377383, -1.00562414]])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

# Models Creation

In [21]:
# model = "adaboost"
model = "knn"
# model = "random-forest"
# model = "svm-rbf"
# model = "svm-poly"

### Adaboost

In [22]:
if model == "adaboost":
    clf = AdaBoostClassifier(n_estimators=100, random_state=42)
    #Adaboost with RandomForest
    clf2 = AdaBoostClassifier(RandomForestClassifier(n_estimators=100), n_estimators=100)
    print(f"Usando {model}")

### KNN

In [23]:
if model == "knn":
    clf = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')

    # Grid Search
    clf.get_params()
    params = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
    grid_search_cv = GridSearchCV(KNeighborsClassifier(), params, n_jobs=-1, verbose=1)
    print(f"Usando {model}")

Usando knn


### Random Forest

In [24]:
if model == "random-forest":
    clf = RandomForestClassifier(random_state=42, n_estimators=100)

    # Grid Search
    rf_clf = RandomForestClassifier(random_state=42, n_estimators=100)
    params_grid = {"max_depth": [3, None], "min_samples_split": [2, 3, 10], "min_samples_leaf": [1, 3, 10], "bootstrap": [True, False], "criterion": ['gini', 'entropy']}
    grid_search_cv = GridSearchCV(rf_clf, params_grid, n_jobs=-1, cv=5, verbose=1, scoring='accuracy', iid=False)
    print(f"Usando {model}")

### SVM rbf

In [25]:
if model == "svm-rbf":
    clf = svm.SVC(C=0.1, gamma=0.01, kernel='rbf')
    
    # Grid Search
    pipeline = Pipeline([('clf', svm.SVC(kernel='rbf', C=1, gamma=0.1))])
    params = {'clf__C':(0.1, 0.5, 1, 2, 5, 10, 20), 'clf__gamma':(0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1)}
    grid_search_cv = GridSearchCV(pipeline, params, n_jobs=-1, cv=3, verbose=1, scoring='accuracy')
    print(f"Usando {model}")

### SVM Poly

In [26]:
if model == "svm-poly":
    clf = svm.SVC(kernel='poly', degree=3)
    
    # Grid Search
    pipeline = Pipeline([('clf', svm.SVC(kernel='poly'))])
    params = {'clf__C':(0.1, 0.5, 1, 2, 5, 10, 20), 'clf__gamma':(0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1), 'clf__degree': (3, 4, 5)}
    grid_search_cv = GridSearchCV(pipeline, params, n_jobs=-1, cv=3, verbose=1, scoring='accuracy')
    print(f"Usando {model}")

# Model Fit

In [27]:
clf.fit(X_train, y_train)
grid_search_cv.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.0s finished


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             verbose=1)

# Performance Measurement

In [28]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        print("Train Result:\n")
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        print("Accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))

        res = cross_val_score(clf, X_train, y_train, cv=4, scoring='accuracy')
        print("Cross Validation \t")
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        print("Test Result:\n")        
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        
        res = cross_val_score(clf, X_test, y_test, cv=4, scoring='accuracy')
        print("Cross Validation \t")
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))

## Model performance

In [29]:
print_score(clf, X_train, y_train, X_test, y_test, train=True)
print("\n******************************\n")
print_score(clf, X_train, y_train, X_test, y_test, train=False)

Train Result:

Confusion Matrix: 
 [[ 693  158]
 [ 136 1062]]

Accuracy score: 0.8565

Classification Report: 
               precision    recall  f1-score   support

           0       0.84      0.81      0.83       851
           1       0.87      0.89      0.88      1198

    accuracy                           0.86      2049
   macro avg       0.85      0.85      0.85      2049
weighted avg       0.86      0.86      0.86      2049


Cross Validation 	
Average Accuracy: 	 0.5944
Accuracy SD: 		 0.0457

******************************

Test Result:

Confusion Matrix: 
 [[129  96]
 [101 187]]

accuracy score: 0.6160

Classification Report: 
               precision    recall  f1-score   support

           0       0.56      0.57      0.57       225
           1       0.66      0.65      0.65       288

    accuracy                           0.62       513
   macro avg       0.61      0.61      0.61       513
weighted avg       0.62      0.62      0.62       513


Cross Validation 	
Aver

## Grid Search performance

In [30]:
y_test_pred_grid = grid_search_cv.predict(X_test)

print_score(grid_search_cv, X_train, y_train, X_test, y_test, train=True)
print("\n******************************\n")
print_score(grid_search_cv, X_train, y_train, X_test, y_test, train=False)

grid_search_cv.best_estimator_
grid_search_cv.best_estimator_.get_params()
grid_search_cv.best_params_
grid_search_cv.cv_results_

Train Result:

Confusion Matrix: 
 [[ 670  181]
 [ 147 1051]]

Accuracy score: 0.8399

Classification Report: 
               precision    recall  f1-score   support

           0       0.82      0.79      0.80       851
           1       0.85      0.88      0.87      1198

    accuracy                           0.84      2049
   macro avg       0.84      0.83      0.83      2049
weighted avg       0.84      0.84      0.84      2049


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Cross Validation 	
Average Accuracy: 	 0.5925
Accuracy SD: 		 0.0638

******************************

Test Result:

Confusion Matrix: 
 [[123 102]
 [102 186]]



...

accuracy score: 0.6023

Classification Report: 
               precision    recall  f1-score   support

           0       0.55      0.55      0.55       225
           1       0.65      0.65      0.65       288

    accuracy                           0.60       513
   macro avg       0.60      0.60      0.60       513
weighted avg       0.60      0.60      0.60       513


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Cross Validation 	
Average Accuracy: 	 0.6220
Accuracy SD: 		 0.0860


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.0s finished


{'mean_fit_time': array([0.00271616, 0.00367064, 0.00246797, 0.00228696, 0.00278397,
        0.00220318, 0.00208445, 0.00238237, 0.00174322, 0.0014957 ]),
 'std_fit_time': array([7.37319046e-04, 1.33103125e-03, 2.95614372e-04, 1.50152182e-04,
        7.39776859e-04, 2.59334077e-04, 4.46611430e-04, 1.57222632e-03,
        3.98669219e-04, 2.76835080e-05]),
 'mean_score_time': array([0.03333201, 0.06429257, 0.05249858, 0.05098653, 0.04663939,
        0.04414682, 0.03921442, 0.03656101, 0.03818798, 0.02529759]),
 'std_score_time': array([0.00255213, 0.02540898, 0.01304631, 0.00301808, 0.00246312,
        0.00379404, 0.00428019, 0.00306569, 0.0012656 , 0.00287072]),
 'param_n_neighbors': masked_array(data=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 1},
  {'n_neighbors': 2},
  {'n_neighbors': 3},
  {'n_neighbors': 

# Save model to disk

In [31]:
import pickle
# save the model to disk
pickle.dump(grid_search_cv, open('./models/model', 'wb'))
print('Done')

Done
